# 🎯 Notebook 05A: Multiclass vs Binary Classification Decision

**Objective:** Determine whether to use multiclass (3-class) or binary (2-class) classification

**The Problem:**
- Prediabetes class (1) is severely underrepresented
- Even with class weights, recall for prediabetes is typically very low
- Need to test empirically before committing to approach

**What We'll Do:**
1. Analyze class distribution
2. Test multiclass with balanced class weights
3. Evaluate prediabetes recall
4. Test binary approach (healthy vs diabetic)
5. **MAKE DATA-DRIVEN DECISION**

---

## 📦 Step 1: Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    recall_score, precision_score, f1_score,
    balanced_accuracy_score
)
from sklearn.pipeline import Pipeline
from collections import Counter

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
import warnings
warnings.filterwarnings('ignore')

print("✅ Setup complete")

## 📊 Step 2: Load Data & Analyze Classes

In [ ]:
df = pd.read_csv('dataset_A_clean.csv')
X = df.drop('Diabetes_012', axis=1)
y = df['Diabetes_012']

print("=" * 60)
print("CLASS DISTRIBUTION ANALYSIS")
print("=" * 60)

class_counts = y.value_counts().sort_index()
total = len(y)

print("\n📊 Original 3-Class Distribution:")
print("-" * 60)
print(f"Class 0 (No Diabetes): {class_counts[0]:7,} ({class_counts[0]/total*100:5.2f}%)")
print(f"Class 1 (Prediabetes): {class_counts[1]:7,} ({class_counts[1]/total*100:5.2f}%) ← PROBLEM")
print(f"Class 2 (Diabetes):    {class_counts[2]:7,} ({class_counts[2]/total*100:5.2f}%)")
print(f"\nTotal: {total:,}")

# Calculate imbalance ratios
ratio_0_to_1 = class_counts[0] / class_counts[1]
ratio_2_to_1 = class_counts[2] / class_counts[1]

print(f"\n⚠️ Imbalance Ratios:")
print(f"   Class 0 : Class 1 = {ratio_0_to_1:.1f}:1")
print(f"   Class 2 : Class 1 = {ratio_2_to_1:.1f}:1")
print(f"\n💡 Prediabetes is {ratio_0_to_1:.0f}x smaller than No Diabetes!")

In [ ]:
# Visualize class imbalance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
colors = ['green', 'orange', 'red']
labels = ['No Diabetes', 'Prediabetes', 'Diabetes']
ax1.bar(labels, class_counts.values, color=colors, alpha=0.7)
ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
ax1.set_title('Class Distribution (Absolute)', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(class_counts.values):
    ax1.text(i, v + 1000, f'{v:,}', ha='center', fontweight='bold')

# Pie chart
ax2.pie(class_counts.values, labels=labels, colors=colors, autopct='%1.1f%%',
        startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
ax2.set_title('Class Distribution (Relative)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ The tiny orange slice is prediabetes - that's the problem!")

## 🧪 Step 3: Test Multiple Class Weight Strategies (Multiclass)

**Goal:** Show comprehensive imbalance handling approaches

**Strategies to test:**
1. No weights (baseline - model ignores imbalance)
2. Balanced weights (sklearn's automatic balancing)
3. Custom weights (manually tuned to emphasize prediabetes)
4. Computed weights (inverse frequency)
5. SMOTE (Synthetic Minority Oversampling)
6. SMOTE + Balanced weights (hybrid)

In [ ]:
print("=" * 60)
print("MULTICLASS: COMPREHENSIVE WEIGHT STRATEGIES")
print("=" * 60)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Split sizes:")
print(f"   Train: {len(X_train):,}")
print(f"   Test: {len(X_test):,}")

# Check train class distribution
train_dist = Counter(y_train)
print(f"\n📊 Training set distribution:")
for cls in [0, 1, 2]:
    print(f"   Class {cls}: {train_dist[cls]:,} ({train_dist[cls]/len(y_train)*100:.2f}%)")

# Calculate class weights for reference
from sklearn.utils.class_weight import compute_class_weight

# Balanced weights (what sklearn uses)
balanced_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

print(f"\n📊 Sklearn Balanced Weights:")
for cls, weight in enumerate(balanced_weights):
    print(f"   Class {cls}: {weight:.3f}")
print(f"   Prediabetes weight is {balanced_weights[1]/balanced_weights[0]:.1f}x Class 0")

### Strategy 1: No Weights (Baseline)

In [ ]:
print("\n" + "=" * 60)
print("STRATEGY 1: NO CLASS WEIGHTS (Baseline)")
print("=" * 60)

pipeline_no_weights = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        n_jobs=-1
    ))
])

pipeline_no_weights.fit(X_train, y_train)
y_pred_no_weights = pipeline_no_weights.predict(X_test)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_no_weights, 
                          target_names=['No Diabetes', 'Prediabetes', 'Diabetes'],
                          digits=3))

recall_no_weights = recall_score(y_test, y_pred_no_weights, average=None)
print(f"\n📈 Recall by Class:")
print(f"   Class 0 (No Diabetes): {recall_no_weights[0]:.3f}")
print(f"   Class 1 (Prediabetes): {recall_no_weights[1]:.3f} ← Focus")
print(f"   Class 2 (Diabetes): {recall_no_weights[2]:.3f}")

### Strategy 2: Balanced Weights (Sklearn Default)

In [ ]:
print("\n" + "=" * 60)
print("STRATEGY 2: BALANCED CLASS WEIGHTS")
print("=" * 60)
print(f"Weights: {dict(enumerate(balanced_weights))}")

pipeline_balanced = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced',
        random_state=42, 
        n_jobs=-1
    ))
])

pipeline_balanced.fit(X_train, y_train)
y_pred_balanced = pipeline_balanced.predict(X_test)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_balanced, 
                          target_names=['No Diabetes', 'Prediabetes', 'Diabetes'],
                          digits=3))

recall_balanced = recall_score(y_test, y_pred_balanced, average=None)
print(f"\n📈 Recall by Class:")
print(f"   Class 0: {recall_balanced[0]:.3f} (Δ={recall_balanced[0]-recall_no_weights[0]:+.3f})")
print(f"   Class 1: {recall_balanced[1]:.3f} (Δ={recall_balanced[1]-recall_no_weights[1]:+.3f}) ← Improved?")
print(f"   Class 2: {recall_balanced[2]:.3f} (Δ={recall_balanced[2]-recall_no_weights[2]:+.3f})")

### Strategy 3: Custom Weights (Aggressive Prediabetes Emphasis)

In [ ]:
print("\n" + "=" * 60)
print("STRATEGY 3: CUSTOM WEIGHTS (Emphasize Prediabetes)")
print("=" * 60)

# Custom weights: Give prediabetes EVEN MORE weight
# Balanced gives ~20x weight to prediabetes
# Let's try 50x, 100x, and 150x

custom_weight_configs = [
    {0: 1.0, 1: 50.0, 2: 5.0},   # 50x prediabetes
    {0: 1.0, 1: 100.0, 2: 5.0},  # 100x prediabetes
    {0: 1.0, 1: 150.0, 2: 5.0},  # 150x prediabetes
]

custom_results = []

for i, weights in enumerate(custom_weight_configs, 1):
    print(f"\n--- Custom Config {i}: {weights} ---")
    
    pipeline_custom = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(
            n_estimators=100,
            class_weight=weights,
            random_state=42,
            n_jobs=-1
        ))
    ])
    
    pipeline_custom.fit(X_train, y_train)
    y_pred_custom = pipeline_custom.predict(X_test)
    
    recall_custom = recall_score(y_test, y_pred_custom, average=None)
    precision_custom = precision_score(y_test, y_pred_custom, average=None)
    f1_custom = f1_score(y_test, y_pred_custom, average=None)
    
    print(f"Prediabetes Recall: {recall_custom[1]:.3f}")
    print(f"Prediabetes Precision: {precision_custom[1]:.3f}")
    print(f"Prediabetes F1: {f1_custom[1]:.3f}")
    
    custom_results.append({
        'config': f'Custom_{i}',
        'prediabetes_weight': weights[1],
        'recall_0': recall_custom[0],
        'recall_1': recall_custom[1],
        'recall_2': recall_custom[2],
        'precision_1': precision_custom[1],
        'f1_1': f1_custom[1],
        'predictions': y_pred_custom
    })

# Find best custom config
best_custom_idx = np.argmax([r['recall_1'] for r in custom_results])
best_custom = custom_results[best_custom_idx]

print(f"\n🏆 Best Custom Config: {best_custom['config']}")
print(f"   Prediabetes weight: {best_custom['prediabetes_weight']:.0f}x")
print(f"   Prediabetes recall: {best_custom['recall_1']:.3f}")
print(f"   Prediabetes precision: {best_custom['precision_1']:.3f}")

### Strategy 4: SMOTE (Synthetic Oversampling)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

print("\n" + "=" * 60)
print("STRATEGY 4: SMOTE (Synthetic Minority Oversampling)")
print("=" * 60)

# SMOTE with different sampling strategies
smote_strategies = [
    ('auto', 'Auto (balance all)'),
    ({0: len(y_train[y_train==0]), 
      1: len(y_train[y_train==0])//5,  # Make prediabetes 1/5 of class 0
      2: len(y_train[y_train==2])}, 'Conservative (1:5:1 ratio)')
]

smote_results = []

for strategy, description in smote_strategies:
    print(f"\n--- SMOTE: {description} ---")
    
    pipeline_smote = ImbPipeline([
        ('scaler', StandardScaler()),
        ('smote', SMOTE(sampling_strategy=strategy, random_state=42)),
        ('classifier', RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            n_jobs=-1
        ))
    ])
    
    pipeline_smote.fit(X_train, y_train)
    y_pred_smote = pipeline_smote.predict(X_test)
    
    recall_smote = recall_score(y_test, y_pred_smote, average=None)
    precision_smote = precision_score(y_test, y_pred_smote, average=None)
    f1_smote = f1_score(y_test, y_pred_smote, average=None)
    
    print(f"Prediabetes Recall: {recall_smote[1]:.3f}")
    print(f"Prediabetes Precision: {precision_smote[1]:.3f}")
    print(f"Prediabetes F1: {f1_smote[1]:.3f}")
    
    smote_results.append({
        'strategy': description,
        'recall_0': recall_smote[0],
        'recall_1': recall_smote[1],
        'recall_2': recall_smote[2],
        'precision_1': precision_smote[1],
        'f1_1': f1_smote[1],
        'predictions': y_pred_smote
    })

# Use the auto SMOTE results for comparison
recall_smote_auto = smote_results[0]['recall_1']
y_pred_smote_auto = smote_results[0]['predictions']

### Strategy 5: SMOTE + Balanced Weights (Hybrid)

In [ ]:
print("\n" + "=" * 60)
print("STRATEGY 5: SMOTE + BALANCED WEIGHTS (Hybrid)")
print("=" * 60)

pipeline_hybrid = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',  # ← Both SMOTE AND weights
        random_state=42,
        n_jobs=-1
    ))
])

pipeline_hybrid.fit(X_train, y_train)
y_pred_hybrid = pipeline_hybrid.predict(X_test)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_hybrid, 
                          target_names=['No Diabetes', 'Prediabetes', 'Diabetes'],
                          digits=3))

recall_hybrid = recall_score(y_test, y_pred_hybrid, average=None)
print(f"\n📈 Recall by Class:")
print(f"   Class 0: {recall_hybrid[0]:.3f}")
print(f"   Class 1: {recall_hybrid[1]:.3f} ← Best so far?")
print(f"   Class 2: {recall_hybrid[2]:.3f}")

## 📊 Step 4: Compare All Multiclass Strategies

In [ ]:
print("=" * 60)
print("MULTICLASS STRATEGY COMPARISON")
print("=" * 60)

# Compile all results
comparison = pd.DataFrame([
    {
        'Strategy': 'No Weights',
        'Recall_NoDiabetes': recall_no_weights[0],
        'Recall_Prediabetes': recall_no_weights[1],
        'Recall_Diabetes': recall_no_weights[2],
        'Macro_Recall': recall_no_weights.mean()
    },
    {
        'Strategy': 'Balanced Weights',
        'Recall_NoDiabetes': recall_balanced[0],
        'Recall_Prediabetes': recall_balanced[1],
        'Recall_Diabetes': recall_balanced[2],
        'Macro_Recall': recall_balanced.mean()
    },
    {
        'Strategy': f'Custom (Best: {best_custom["prediabetes_weight"]:.0f}x)',
        'Recall_NoDiabetes': best_custom['recall_0'],
        'Recall_Prediabetes': best_custom['recall_1'],
        'Recall_Diabetes': best_custom['recall_2'],
        'Macro_Recall': (best_custom['recall_0'] + best_custom['recall_1'] + best_custom['recall_2'])/3
    },
    {
        'Strategy': 'SMOTE (Auto)',
        'Recall_NoDiabetes': smote_results[0]['recall_0'],
        'Recall_Prediabetes': smote_results[0]['recall_1'],
        'Recall_Diabetes': smote_results[0]['recall_2'],
        'Macro_Recall': (smote_results[0]['recall_0'] + smote_results[0]['recall_1'] + smote_results[0]['recall_2'])/3
    },
    {
        'Strategy': 'SMOTE + Balanced',
        'Recall_NoDiabetes': recall_hybrid[0],
        'Recall_Prediabetes': recall_hybrid[1],
        'Recall_Diabetes': recall_hybrid[2],
        'Macro_Recall': recall_hybrid.mean()
    }
]).sort_values('Recall_Prediabetes', ascending=False)

print("\n📊 Results Ranked by Prediabetes Recall:")
print(comparison.to_string(index=False))

# Highlight best
best_strategy = comparison.iloc[0]
print(f"\n🏆 Best Strategy for Prediabetes: {best_strategy['Strategy']}")
print(f"   Prediabetes Recall: {best_strategy['Recall_Prediabetes']:.3f}")
print(f"   Diabetes Recall: {best_strategy['Recall_Diabetes']:.3f}")
print(f"   Macro Recall: {best_strategy['Macro_Recall']:.3f}")

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(comparison))
width = 0.25

ax.bar(x - width, comparison['Recall_NoDiabetes'], width, 
       label='No Diabetes', color='green', alpha=0.7)
ax.bar(x, comparison['Recall_Prediabetes'], width, 
       label='Prediabetes', color='orange', alpha=0.7)
ax.bar(x + width, comparison['Recall_Diabetes'], width, 
       label='Diabetes', color='red', alpha=0.7)

ax.set_ylabel('Recall', fontsize=12, fontweight='bold')
ax.set_title('Multiclass: Recall by Strategy', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison['Strategy'], rotation=15, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.axhline(y=0.30, color='black', linestyle='--', alpha=0.5, label='Acceptable (0.30)')

plt.tight_layout()
plt.show()

print("\n💡 Look at the orange bars (Prediabetes) - are ANY above 0.30?")

## 🔄 Step 4: Test Binary Classification

**Approach:** Combine classes 0 (No Diabetes) and 1 (Prediabetes) → Healthy (0)

vs Class 2 (Diabetes) → Diabetic (1)

In [ ]:
print("=" * 60)
print("TESTING BINARY CLASSIFICATION")
print("=" * 60)

# Create binary target
y_binary = (y == 2).astype(int)  # 0 = healthy/prediabetes, 1 = diabetes

print("\n📊 Binary Class Distribution:")
binary_counts = y_binary.value_counts().sort_index()
print(f"   Class 0 (Healthy + Prediabetes): {binary_counts[0]:,} ({binary_counts[0]/len(y_binary)*100:.2f}%)")
print(f"   Class 1 (Diabetes):              {binary_counts[1]:,} ({binary_counts[1]/len(y_binary)*100:.2f}%)")
print(f"\n   Imbalance ratio: {binary_counts[0]/binary_counts[1]:.1f}:1")
print(f"   This is MUCH more manageable than 41:1!")

In [ ]:
# Split binary data
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# Train binary classifier
pipeline_binary = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

pipeline_binary.fit(X_train_bin, y_train_bin)
y_pred_binary = pipeline_binary.predict(X_test_bin)

print("\n📊 Classification Report (Binary):")
print(classification_report(y_test_bin, y_pred_binary,
                          target_names=['Healthy', 'Diabetic'],
                          digits=3))

# Get recall for diabetes
recall_diabetes_binary = recall_score(y_test_bin, y_pred_binary)
print(f"\n📈 Diabetes Recall (Binary): {recall_diabetes_binary:.3f}")

In [ ]:
# Binary confusion matrix
cm_binary = confusion_matrix(y_test_bin, y_pred_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_binary, annot=True, fmt='d', cmap='Greens',
           xticklabels=['Healthy', 'Diabetic'],
           yticklabels=['Healthy', 'Diabetic'])
plt.title('Binary Classification Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

## 📊 Step 5: Comparison & Decision

In [ ]:
print("=" * 60)
print("COMPREHENSIVE COMPARISON")
print("=" * 60)

# Calculate key metrics for all approaches
results = pd.DataFrame([
    {
        'Approach': 'Multiclass (No Weights)',
        'Classes': 3,
        'Prediabetes Recall': recall_prediabetes_no_weights,
        'Diabetes Recall': recall_score(y_test, y_pred_no_weights, labels=[2], average=None)[0],
        'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_no_weights),
        'Macro F1': f1_score(y_test, y_pred_no_weights, average='macro')
    },
    {
        'Approach': 'Multiclass (Balanced Weights)',
        'Classes': 3,
        'Prediabetes Recall': recall_prediabetes_balanced,
        'Diabetes Recall': recall_score(y_test, y_pred_balanced, labels=[2], average=None)[0],
        'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred_balanced),
        'Macro F1': f1_score(y_test, y_pred_balanced, average='macro')
    },
    {
        'Approach': 'Binary',
        'Classes': 2,
        'Prediabetes Recall': np.nan,  # Not applicable
        'Diabetes Recall': recall_diabetes_binary,
        'Balanced Accuracy': balanced_accuracy_score(y_test_bin, y_pred_binary),
        'Macro F1': f1_score(y_test_bin, y_pred_binary, average='macro')
    }
])

print("\n📊 Results Summary:")
print(results.to_string(index=False))

print("\n" + "=" * 60)
print("KEY FINDINGS")
print("=" * 60)

print(f"\n1. Prediabetes Detection (Multiclass):")
print(f"   Without weights: {recall_prediabetes_no_weights:.1%}")
print(f"   With weights: {recall_prediabetes_balanced:.1%}")
if recall_prediabetes_balanced < 0.30:
    print(f"   ❌ Still POOR - can't reliably detect prediabetes!")
elif recall_prediabetes_balanced < 0.50:
    print(f"   ⚠️ WEAK - misses >50% of prediabetes cases")
else:
    print(f"   ✅ ACCEPTABLE - might be viable")

print(f"\n2. Diabetes Detection:")
diabetes_recall_multi = recall_score(y_test, y_pred_balanced, labels=[2], average=None)[0]
print(f"   Multiclass: {diabetes_recall_multi:.1%}")
print(f"   Binary: {recall_diabetes_binary:.1%}")
if recall_diabetes_binary > diabetes_recall_multi:
    print(f"   ✅ Binary is BETTER by {(recall_diabetes_binary - diabetes_recall_multi)*100:.1f} percentage points")

print(f"\n3. Overall Performance:")
print(f"   Multiclass Macro F1: {f1_score(y_test, y_pred_balanced, average='macro'):.3f}")
print(f"   Binary Macro F1: {f1_score(y_test_bin, y_pred_binary, average='macro'):.3f}")

## 🎯 Step 6: Final Decision

In [ ]:
print("=" * 60)
print("FINAL DECISION")
print("=" * 60)

# Decision logic
if recall_prediabetes_balanced < 0.30:
    decision = "BINARY"
    reason = f"Prediabetes recall ({recall_prediabetes_balanced:.1%}) is too low to be clinically useful"
elif recall_prediabetes_balanced < 0.50:
    decision = "BINARY"
    reason = f"Prediabetes recall ({recall_prediabetes_balanced:.1%}) misses majority of cases"
else:
    if recall_diabetes_binary > diabetes_recall_multi + 0.05:
        decision = "BINARY"
        reason = f"Binary achieves better diabetes detection with simpler model"
    else:
        decision = "MULTICLASS"
        reason = f"Prediabetes recall ({recall_prediabetes_balanced:.1%}) is acceptable"

print(f"\n🎯 DECISION: Use {decision} Classification")
print(f"\n📋 Reasoning: {reason}")

if decision == "BINARY":
    print(f"\n✅ Binary Classification Benefits:")
    print(f"   1. Better class balance ({binary_counts[0]/binary_counts[1]:.1f}:1 vs 41:1)")
    print(f"   2. Higher diabetes recall ({recall_diabetes_binary:.1%})")
    print(f"   3. Simpler model (2 classes vs 3)")
    print(f"   4. Easier to optimize with SMOTE/sampling")
    print(f"   5. Clinically relevant (diabetes vs non-diabetes)")
    
    print(f"\n⚠️ Trade-off:")
    print(f"   - Cannot specifically identify prediabetes")
    print(f"   - But prediabetes was undetectable anyway in multiclass!")
else:
    print(f"\n✅ Multiclass Classification Benefits:")
    print(f"   1. Can identify all three groups")
    print(f"   2. Prediabetes recall is acceptable ({recall_prediabetes_balanced:.1%})")
    print(f"   3. More granular risk stratification")

print(f"\n🚀 NEXT STEPS:")
if decision == "BINARY":
    print(f"   → Proceed with binary classification (Healthy vs Diabetic)")
    print(f"   → Test sampling strategies (SMOTE, ADASYN, etc.)")
    print(f"   → Optimize for diabetes recall")
else:
    print(f"   → Proceed with multiclass classification")
    print(f"   → Test custom class weights for prediabetes")
    print(f"   → Optimize for balanced recall across all classes")

# Save decision
with open('classification_decision.txt', 'w') as f:
    f.write(f"Decision: {decision}\n")
    f.write(f"Reason: {reason}\n")
    if decision == "BINARY":
        f.write(f"Target: y_binary = (y == 2).astype(int)\n")
    else:
        f.write(f"Target: y (3 classes)\n")

print(f"\n✅ Decision saved to classification_decision.txt")

## 📝 Summary for Report

In [ ]:
print("=" * 60)
print("FOR YOUR REPORT")
print("=" * 60)

print("""
### Classification Approach Selection

**Problem Statement:**
The dataset contains three classes: No Diabetes (83.5%), Prediabetes (2.0%), 
and Diabetes (14.5%), presenting a severe class imbalance of 41:1 for the 
minority prediabetes class.

**Multiclass vs Binary Evaluation:**

We empirically tested both approaches using Random Forest with balanced class weights:

""")

if decision == "BINARY":
    print(f"""
1. **Multiclass (3-class) Results:**
   - Prediabetes recall: {recall_prediabetes_balanced:.1%}
   - Diabetes recall: {diabetes_recall_multi:.1%}
   - Interpretation: Even with balanced class weights, prediabetes detection 
     was unreliable, missing {(1-recall_prediabetes_balanced)*100:.0f}% of cases.

2. **Binary (2-class) Results:**
   - Diabetes recall: {recall_diabetes_binary:.1%}
   - Balanced accuracy: {balanced_accuracy_score(y_test_bin, y_pred_binary):.1%}
   - Class imbalance: {binary_counts[0]/binary_counts[1]:.1f}:1 (much more manageable)

**Decision: Binary Classification**

We selected binary classification (Healthy vs Diabetic) because:
1. Prediabetes class is too small for reliable multiclass detection
2. Binary approach achieves {(recall_diabetes_binary - diabetes_recall_multi)*100:+.1f} 
   percentage point improvement in diabetes recall
3. Reduced imbalance ratio ({binary_counts[0]/binary_counts[1]:.1f}:1 vs 41:1) 
   enables more effective sampling strategies
4. Clinical relevance: Primary goal is diabetes identification

**Justification:**
While losing prediabetes granularity is a trade-off, the multiclass approach 
failed to reliably detect it anyway. Binary classification provides better 
diabetes detection with a simpler, more robust model.
""")
else:
    print(f"""
**Decision: Multiclass Classification**

Despite severe class imbalance, multiclass classification achieved acceptable 
prediabetes recall ({recall_prediabetes_balanced:.1%}) with balanced class weights. 
This enables:
1. Granular risk stratification (three-tier)
2. Identification of at-risk prediabetes population
3. Targeted interventions for each group

We will use custom class weights and advanced sampling to further improve 
prediabetes detection.
""")

print("\n" + "=" * 60)